In [ ]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import os
import yaml
from PIL import Image

import torch
import torch.nn as nn

from net.basenet import GaussNet3, GaussianConv2d
from dataprocess.sirst import NUDTDataset, IRSTD1kDataset

In [ ]:
base_size = 128
pict = np.array(cv2.resize(cv2.imread('W:/DataSets/ISTD/IRSTD-1k/trainval/masks/XDU116.png', 0), [base_size, base_size], interpolation=cv2.INTER_LINEAR))
pict = np.array((pict, np.array(cv2.resize(cv2.imread('W:/DataSets/ISTD/IRSTD-1k/trainval/images/XDU116.png', 0), [base_size, base_size], interpolation=cv2.INTER_LINEAR))))
# pict = cv2.resize(pict, [2, 256, 256], interpolation=cv2.INTER_LINEAR)
pict = torch.from_numpy(pict).type(torch.float32)/255
# pict = pict.unsqueeze(0).view(2, 1, 240, 320)
# pict = pict[:,:,:,:240]
# pict = pict[:,:,:240]
print(pict.shape)

In [ ]:
net = GaussianConv2d(1, 3, 0.6)

In [ ]:
res = net(pict[1:2,])

print(net.atten.weight.data)
print(net.tempreture)
print(net.bias)

originp = np.array(pict)
# originp = (originp * 255).astype(np.uint8)
gaussp = res.detach().numpy()
gaussp = (gaussp - np.min(gaussp)) / (np.max(gaussp)-np.min(gaussp))
# gaussp = (gaussp * 255).astype(np.uint8)

# # 转换为PIL图像
# pil_image = Image.fromarray(gaussp[0, 0])

# # 保存图像
# pil_image.save("output4.png")

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
# plt.imshow(gaussp[0,0], cmap='gray')
axes[0].imshow(gaussp[0, 0], cmap='gray')
axes[0].axis('off')  # 关闭坐标轴显示
    
axes[1].imshow(originp[1], cmap='gray')
axes[1].axis('off')  # 同样关闭坐标轴显示
# plt.title('Blurred Image')
plt.tight_layout()
plt.show()

In [ ]:
import math

def gaussian_kernel(channel, kernel_size, sigma):
    """创建一个高斯核"""
    # 创建一个一维高斯核
    kernel_1d = torch.Tensor([math.exp(-z ** 2.0 / (2 * sigma ** 2)) for z in range(-kernel_size // 2 + 1, kernel_size // 2 + 1)])
    # 将一维高斯核扩展为二维
    kernel_2d = torch.outer(kernel_1d, kernel_1d)
    # 归一化
    kernel_2d /= kernel_2d.sum()
    kernel_2d[kernel_size // 2, kernel_size // 2] = -kernel_2d[kernel_size // 2, kernel_size // 2]
    # 整个卷积核参数
    params = torch.zeros((channel, channel, kernel_size, kernel_size))
    for i in range(channel):
        params[i, i] = kernel_2d
    return params

res = gaussian_kernel(1,3,0.6)
print(res)

In [ ]:
pt_path = "result/20240819T08-38-33_new_gauss2_1_irstd1k/best.pkl"
params = torch.load(pt_path)
# for key in params:
#     print(key)
for k in params:
    if "gauss.bias" in k:
        print(k)
        print(params[k])

In [ ]:
cfg_path = 'cfg.yaml'
with open(cfg_path) as f:
    cfg = yaml.safe_load(f)

net = GaussNet3(1, cfg)

In [ ]:
from torchvision import transforms

augment = transforms.Compose([
                    transforms.RandomAffine(degrees=180, translate=(0.1, 0.1), shear=0),
                ])

In [ ]:
a = np.array(cv2.imread("result/20240812T12-46-54_gauss9_1_irstd1k/nudt_result.png", 0))

In [ ]:
print(a.shape)

In [ ]:
import math
import torch
sigma = 2
kernel_size = 5

In [ ]:
def generate_gauss_kernel():
    kernel_size = 3
    sigma = 2

    kernels = []
    for i in range(kernel_size):
        for j in range(kernel_size):
            x = torch.arange(0, kernel_size, 1, dtype=torch.float32) - i
            y = torch.arange(0, kernel_size, 1, dtype=torch.float32) - j
            kernel_1d_x = torch.exp(-(x**2) / (2 * sigma**2))
            kernel_1d_y = torch.exp(-(y**2) / (2 * sigma**2))
            kernel_2d = torch.outer(kernel_1d_x, kernel_1d_y)
            # 归一化
            kernel_2d /= kernel_2d.sum()
            kernels.append(kernel_2d)

    return kernels

result = generate_gauss_kernel()
for i in result:
    print(i)

In [ ]:
from dataprocess.sirst import IRSTD1kDataset
import yaml
import torch

cfg_path = 'cfg.yaml'
with open(cfg_path) as f:
    cfg = yaml.safe_load(f)
print(cfg["multiscalefeature_outchannel"])

dataset = IRSTD1kDataset(base_dir=r"W:/DataSets/ISTD/IRSTD-1k", mode="train", base_size=256, multiscale_lable=True, cfg=cfg)
# train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, pin_memory=True)

In [ ]:
i = 8
labels = []
multiscale_labels = []
for data, label, multiscale_label in dataset:
    i -= 1
    if i < 0:
        break
    labels.append(label.squeeze())
    multiscale_labels.append(multiscale_label[0].squeeze())

In [ ]:
label = np.array(labels)
multiscale_label = np.array(multiscale_labels)
print(multiscale_label.shape)
# originp = (originp * 255).astype(np.uint8)
# gaussp = res.detach().numpy()
# gaussp = (gaussp - np.min(gaussp)) / (np.max(gaussp)-np.min(gaussp))
# gaussp = (gaussp * 255).astype(np.uint8)

# # 转换为PIL图像
# pil_image = Image.fromarray(gaussp[0, 0])

# # 保存图像
# pil_image.save("output4.png")
row_num = label.shape[0]
fig, axes = plt.subplots(row_num, 2)
for i in range(row_num):
    # plt.imshow(gaussp[0,0], cmap='gray')
    axes[i, 0].imshow(label[i], cmap='gray')
    axes[i, 0].axis('off')  # 关闭坐标轴显示
        
    axes[i, 1].imshow(multiscale_label[i], cmap='gray')
    axes[i, 1].axis('off')  # 同样关闭坐标轴显示
    # plt.title('Blurred Image')
plt.tight_layout()
plt.show()

In [ ]:
print(multiscale_label[0])

In [ ]:
import torch
from net.twotasknet import Heatmap_net

# 1. 准备模型
model = Heatmap_net(1, cfg)
model.eval()  # 设置模型为评估模式

# 2. 定义输入形状
dummy_input = torch.randn(32, 1, 256, 256)  # 一个batch_size=1的示例输入

# 3. 导出模型 
torch.onnx.export(
    model,  # 模型实例
    dummy_input,  # 示例输入
    "heatmap_net.onnx",  # 输出文件名
    export_params=True,  # 存储模型参数 
    opset_version=17,  # ONNX版本
    do_constant_folding=True,  # 是否执行常量折叠优化
    input_names=['input'],  # 输入节点名称
    output_names=['output'],  # 输出节点名称
    dynamic_axes={'input': {0: 'batch_size'},  # 可变维度
                  'output': {0: 'batch_size'}}
)

In [1]:
from net.twotasknet import HeatMaptoImg
import torch

In [2]:
img = torch.randn((4, 1, 256, 256))
heatmap = [ (torch.randn(4, 1, 8, 8)-0.8)>0,
            (torch.randn(4, 1, 16, 16)-0.8)>0,
            (torch.randn(4, 1, 32, 32)-0.8)>0]

res = HeatMaptoImg(heatmap, img)

In [3]:
for i in res:
    for j in i:
        for z in j:
            print(z.shape)

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([16, 16])
torch.Size([1

In [1]:
from dataprocess.croped_sirst import Crop_IRSTD1kDataset

dataset = Crop_IRSTD1kDataset(base_dir=r"W:/DataSets/ISTD/IRSTD-1k", mode="train", base_size=16)

In [4]:
from net.twotasknet import UnitLabels

In [ ]:
labels8 = [[torch.randn(4,8,8) > 0]for i in range(8)]
labels16 = [[torch.randn(6,16,16) > 0]for i in range(8)]
labels32 = [[torch.randn(2,32,32) > 0]for i in range(8)]
multiscale_labels = [labels8, labels16, labels32]



In [2]:
import torch

print(torch.randint(0,1, (2,4,4)))

tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]])


In [4]:
a = torch.tensor([1,2,3])
b = a[1:1]
print(b.shape)

torch.Size([0])
